In [9]:
import matplotlib

matplotlib.use('Agg')
import train_P as train
import dataset as ds
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

import utils
import os
import conll2brat
import glob
import codecs
import shutil
import time
import copy
import evaluate
import random
import pickle
import brat2conll
import numpy as np
import utils_nlp
import distutils.util as distutils_util
import configparser
from pprint import pprint

In [13]:
parameters = {'pretrained_model_folder':'../model',
                      'dataset_text_folder':'../../../ML_EntityData/data/en',
                      'character_embedding_dimension':25,
                      'character_lstm_hidden_state_dimension':25,
                      'check_for_digits_replaced_with_zeros':True,
                      'check_for_lowercase':True,
                      'debug':False,
                      'dropout_rate':0.5,
                      'experiment_name':'test',
                      'freeze_token_embeddings':False,
                      'gradient_clipping_value':5.0,
                      'learning_rate':0.005,
                      'load_only_pretrained_token_embeddings':False,
                      'load_all_pretrained_token_embeddings':False,
#                       'main_evaluation_mode':'conll',
                      'maximum_number_of_epochs':500,
                      'number_of_cpu_threads':8,
                      'number_of_gpus':0,
                      'optimizer':'sgd',
                      'output_folder':'../../../ML_EntityData/output',
                      'patience':10,
#                       'plot_format':'pdf',
                      'reload_character_embeddings':True,
                      'reload_character_lstm':True,
                      'reload_crf':True,
                      'reload_feedforward':True,
                      'reload_token_embeddings':True,
                      'reload_token_lstm':True,
                      'remap_unknown_tokens_to_unk':True,
#                       'spacylanguage':'en',
                      'tagging_format':'bioes',
                      'token_embedding_dimension':100,
                      'token_lstm_hidden_state_dimension':100,
                      'token_pretrained_embedding_filepath':'../../../ML_EntityData/embedding/en/glove.6B.100d.txt',
                      'tokenizer':'spacy',
                      'train_model':True,
                      'use_character_lstm':True,
#                       'use_crf':True,
                      'use_pretrained_model':False,
                      'verbose':False}

In [4]:
import utils
import utils_data as ds
# Load dataset
dataset_filepaths, dataset_brat_folders = utils.get_valid_dataset_filepaths(parameters)
dataset = ds.DatasetP(verbose=False, debug=False)
token_to_vector = dataset.load_dataset(dataset_filepaths, parameters)

Checking the validity of BRAT-formatted train set... Done.
Checking compatibility between CONLL and BRAT for train_compatible_with_brat set ... Done.
Checking validity of CONLL BIOES format... Done.
Checking the validity of BRAT-formatted valid set... Done.
Checking compatibility between CONLL and BRAT for valid_compatible_with_brat set ... Done.
Checking validity of CONLL BIOES format... Done.
Checking the validity of BRAT-formatted test set... Done.
Checking compatibility between CONLL and BRAT for test_compatible_with_brat set ... Done.
Checking validity of CONLL BIOES format... Done.
Load dataset... done (73.73 seconds)


In [5]:
import tensorflow as tf
from BLSTM_CRF_P import BLSTM_CRF
# Create model lstm+crf
session_conf = tf.ConfigProto(
            intra_op_parallelism_threads=parameters['number_of_cpu_threads'],
            inter_op_parallelism_threads=parameters['number_of_cpu_threads'],
            device_count={'CPU': 2, 'GPU': parameters['number_of_gpus']},
            allow_soft_placement=True,
            # automatically choose an existing and supported device to run the operations in case the specified one doesn't exist
            log_device_placement=False
        )
sess = tf.Session(config=session_conf)

with sess.as_default():
    # Create model and initialize or load pretrained model
    ### Instantiate the model
    model = BLSTM_CRF(dataset, token_embedding_dimension=parameters['token_embedding_dimension'],
                       character_lstm_hidden_state_dimension=parameters['character_lstm_hidden_state_dimension'],
                       token_lstm_hidden_state_dimension=parameters['token_lstm_hidden_state_dimension'],
                       character_embedding_dimension=parameters['character_embedding_dimension'],
                       gradient_clipping_value=parameters['gradient_clipping_value'],
                       learning_rate=parameters['learning_rate'],
                       freeze_token_embeddings=parameters['freeze_token_embeddings'],
                       optimizer=parameters['optimizer'],
                       maximum_number_of_epochs=parameters['maximum_number_of_epochs'])

sess.run(tf.global_variables_initializer())

embedded_characters: Tensor("character_embedding/embedded_characters:0", shape=(?, ?, 25), dtype=float32)
embedded_tokens: Tensor("token_embedding/embedding_lookup:0", shape=(?, 100), dtype=float32)
token_lstm_input: Tensor("concatenate_token_and_character_vectors/token_lstm_input:0", shape=(?, 163), dtype=float32)
token_lstm_input_drop: Tensor("dropout/token_lstm_input_drop/mul:0", shape=(?, 163), dtype=float32)
token_lstm_input_drop_expanded: Tensor("dropout/token_lstm_input_drop_expanded:0", shape=(1, ?, 163), dtype=float32)
unary_scores_expanded: Tensor("crf/unary_scores_expanded:0", shape=(1, ?, 19), dtype=float32)
input_label_indices_flat_batch: Tensor("crf/input_label_indices_flat_batch:0", shape=(1, ?), dtype=int32)
sequence_lengths: Tensor("crf/sequence_lengths:0", shape=(1,), dtype=int32)


d:\programs\python\python36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [6]:
import numpy as np
model.load_pretrained_token_embeddings(sess, dataset,embedding_filepath=parameters['token_pretrained_embedding_filepath'],
                                                       check_lowercase= parameters['check_for_lowercase'],check_digits=parameters['check_for_digits_replaced_with_zeros'],
                                                       token_to_vector=token_to_vector)
# Initial params_train
transition_params_trained = np.random.rand(len(dataset.unique_labels) + 2,len(dataset.unique_labels) + 2)

del token_to_vector

Load token embeddings... done (0.22 seconds)
number_of_token_original_case_found: 14618
number_of_token_lowercase_found: 11723
number_of_token_digits_replaced_with_zeros_found: 119
number_of_token_lowercase_and_digits_replaced_with_zeros_found: 16
number_of_loaded_word_vectors: 26476
dataset.vocabulary_size: 28984


In [7]:
import time
import copy
import os
import pickle
stats_graph_folder, experiment_timestamp = utils.create_stats_graph_folder(parameters)

        # Initialize and save execution details
start_time = time.time()
# results = {}
# results['epoch'] = {}
# results['execution_details'] = {}
# results['execution_details']['train_start'] = start_time
# results['execution_details']['time_stamp'] = experiment_timestamp
# results['execution_details']['early_stop'] = False
# results['execution_details']['keyboard_interrupt'] = False
# results['execution_details']['num_epochs'] = 0
# results['model_options'] = copy.copy(parameters)

model_folder = os.path.join(stats_graph_folder, 'model')
utils.create_folder_if_not_exists(model_folder)

pickle.dump(dataset, open(os.path.join(model_folder, 'dataset.pickle'), 'wb'))

In [19]:
bad_counter = 0  # number of epochs with no improvement on the validation test in terms of F1-score
previous_best_valid_f1_score = 0
epoch_number = -1

while True:

    step = 0
    epoch_number += 1
    print('\nStarting epoch {0}'.format(epoch_number))

    epoch_start_time = time.time()

    if epoch_number != 0:
        # Train model: loop over all sequences of training set with shuffling
        sequence_numbers = list(range(len(dataset.token_indices['train'])))
        random.shuffle(sequence_numbers)
        for sequence_number in sequence_numbers:
            transition_params_trained = train.train_step(sess, dataset, sequence_number, model, parameters['dropout_rate'])
            step += 1
            if step % 10 == 0:
                print('Training {0:.2f}% done'.format(step / len(sequence_numbers) * 100), end='\r', flush=True)

    epoch_elapsed_training_time = time.time() - epoch_start_time
    print('Training completed in {0:.2f} seconds'.format(epoch_elapsed_training_time), flush=True)

    y_pred, y_true, output_filepaths = train.predict_labels(sess=sess,model= model,transition_params_trained= transition_params_trained,
                                                                         dataset=dataset,epoch_number= epoch_number,
                                                                        stats_graph_folder= stats_graph_folder,dataset_filepaths= dataset_filepaths,
                                                                        tagging_format= parameters['tagging_format'])

       
#     if epoch_number % 3 ==0:
    model.saver.save(sess, os.path.join(model_folder, 'model_{0:05d}.ckpt'.format(epoch_number%3)))
        
    if epoch_number > 20 :
        break


Starting epoch 0
Training completed in 0.00 seconds
Evaluate model on the train set
             precision    recall  f1-score   support

      B-LOC     0.9474    0.9520    0.9497      1041
     B-MISC     0.9071    0.9336    0.9202       858
      B-ORG     0.9705    0.9412    0.9557      2485
      B-PER     0.9657    0.9923    0.9788      4284
      E-LOC     0.9457    0.9529    0.9493      1041
     E-MISC     0.9106    0.9382    0.9242       858
      E-ORG     0.9664    0.9368    0.9514      2485
      E-PER     0.9700    0.9965    0.9831      4284
      I-LOC     0.8496    0.8276    0.8384       116
     I-MISC     0.8947    0.8586    0.8763       297
      I-ORG     0.9501    0.9221    0.9359      1219
      I-PER     0.8824    0.9836    0.9302       244
          O     0.0000    0.0000    0.0000       406
      S-LOC     0.9745    0.9831    0.9788      6099
     S-MISC     0.9535    0.9384    0.9459      2580
      S-ORG     0.9733    0.9494    0.9612      3836
      S-PER  


Evaluate model on the test set
             precision    recall  f1-score   support

      B-LOC     0.7600    0.9009    0.8245       232
     B-MISC     0.6223    0.6610    0.6411       177
      B-ORG     0.8536    0.8359    0.8447       579
      B-PER     0.9809    0.9908    0.9858      1086
      E-LOC     0.7564    0.8966    0.8205       232
     E-MISC     0.6508    0.6949    0.6721       177
      E-ORG     0.8622    0.8428    0.8524       579
      E-PER     0.9799    0.9890    0.9844      1086
      I-LOC     0.6000    0.7200    0.6545        25
     I-MISC     0.6250    0.5128    0.5634        39
      I-ORG     0.7634    0.8320    0.7963       256
      I-PER     0.9437    0.9571    0.9504        70
          O     0.0000    0.0000    0.0000       282
      S-LOC     0.9161    0.9429    0.9293      1436
     S-MISC     0.8631    0.8286    0.8455       525
      S-ORG     0.9206    0.8678    0.8934      1082
      S-PER     0.9173    0.8776    0.8970       531

avg / total 


Evaluate model on the valid set
             precision    recall  f1-score   support

      B-LOC     0.8996    0.9573    0.9275       234
     B-MISC     0.8273    0.8016    0.8142       257
      B-ORG     0.8614    0.8422    0.8517       450
      B-PER     0.9887    0.9887    0.9887      1234
      E-LOC     0.8992    0.9530    0.9253       234
     E-MISC     0.8440    0.8210    0.8323       257
      E-ORG     0.8841    0.8644    0.8742       450
      E-PER     0.9862    0.9862    0.9862      1234
      I-LOC     0.7826    0.7826    0.7826        23
     I-MISC     0.7681    0.5955    0.6709        89
      I-ORG     0.8171    0.9203    0.8656       301
      I-PER     0.9811    0.7123    0.8254        73
          O     0.0000    0.0000    0.0000       137
      S-LOC     0.9644    0.9807    0.9725      1603
     S-MISC     0.9349    0.9068    0.9206       665
      S-ORG     0.9502    0.9214    0.9356       891
      S-PER     0.8979    0.9260    0.9117       608

avg / total



Starting epoch 8
Training completed in 425.67 seconds
Evaluate model on the train set
             precision    recall  f1-score   support

      B-LOC     0.9824    0.9645    0.9733      1041
     B-MISC     0.9168    0.9767    0.9458       858
      B-ORG     0.9727    0.9738    0.9733      2485
      B-PER     0.9941    0.9911    0.9926      4284
      E-LOC     0.9805    0.9654    0.9729      1041
     E-MISC     0.9223    0.9825    0.9515       858
      E-ORG     0.9715    0.9734    0.9725      2485
      E-PER     0.9948    0.9918    0.9933      4284
      I-LOC     0.9901    0.8621    0.9217       116
     I-MISC     0.8818    0.9798    0.9282       297
      I-ORG     0.9772    0.9475    0.9621      1219
      I-PER     0.9792    0.9631    0.9711       244
          O     0.0000    0.0000    0.0000       266
      S-LOC     0.9910    0.9802    0.9856      6099
     S-MISC     0.9276    0.9740    0.9503      2580
      S-ORG     0.9761    0.9698    0.9729      3836
      S-PE


Evaluate model on the test set
             precision    recall  f1-score   support

      B-LOC     0.7431    0.9224    0.8231       232
     B-MISC     0.5737    0.6158    0.5940       177
      B-ORG     0.8205    0.8446    0.8323       579
      B-PER     0.9790    0.9871    0.9830      1086
      E-LOC     0.7396    0.9181    0.8192       232
     E-MISC     0.6474    0.6949    0.6703       177
      E-ORG     0.8286    0.8515    0.8399       579
      E-PER     0.9772    0.9853    0.9812      1086
      I-LOC     0.5556    0.8000    0.6557        25
     I-MISC     0.3521    0.6410    0.4545        39
      I-ORG     0.7147    0.8711    0.7852       256
      I-PER     0.9420    0.9286    0.9353        70
          O     0.0000    0.0000    0.0000       382
      S-LOC     0.9321    0.9373    0.9347      1436
     S-MISC     0.8923    0.8362    0.8633       525
      S-ORG     0.9043    0.8993    0.9018      1082
      S-PER     0.9329    0.8908    0.9114       531

avg / total 


Evaluate model on the valid set
             precision    recall  f1-score   support

      B-LOC     0.9061    0.9487    0.9269       234
     B-MISC     0.7674    0.8599    0.8110       257
      B-ORG     0.8988    0.8289    0.8624       450
      B-PER     0.9847    0.9911    0.9879      1234
      E-LOC     0.9057    0.9444    0.9247       234
     E-MISC     0.7766    0.8794    0.8248       257
      E-ORG     0.9159    0.8467    0.8799       450
      E-PER     0.9839    0.9895    0.9867      1234
      I-LOC     0.6957    0.6957    0.6957        23
     I-MISC     0.5714    0.6292    0.5989        89
      I-ORG     0.9032    0.8372    0.8690       301
      I-PER     0.9464    0.7260    0.8217        73
          O     0.0000    0.0000    0.0000       139
      S-LOC     0.9690    0.9757    0.9723      1603
     S-MISC     0.9416    0.8977    0.9192       665
      S-ORG     0.9352    0.9394    0.9373       891
      S-PER     0.9116    0.9326    0.9220       608

avg / total



Starting epoch 16
Training completed in 426.44 seconds
Evaluate model on the train set
             precision    recall  f1-score   support

      B-LOC     0.9913    0.9798    0.9855      1041
     B-MISC     0.9600    0.9802    0.9700       858
      B-ORG     0.9886    0.9771    0.9828      2485
      B-PER     0.9907    0.9979    0.9943      4284
      E-LOC     0.9903    0.9779    0.9841      1041
     E-MISC     0.9613    0.9837    0.9724       858
      E-ORG     0.9898    0.9783    0.9840      2485
      E-PER     0.9914    0.9986    0.9950      4284
      I-LOC     0.9910    0.9483    0.9692       116
     I-MISC     0.9630    0.9630    0.9630       297
      I-ORG     0.9788    0.9828    0.9808      1219
      I-PER     0.9796    0.9836    0.9816       244
          O     0.0000    0.0000    0.0000       151
      S-LOC     0.9957    0.9811    0.9884      6099
     S-MISC     0.9807    0.9651    0.9728      2580
      S-ORG     0.9737    0.9833    0.9785      3836
      S-P


Evaluate model on the test set
             precision    recall  f1-score   support

      B-LOC     0.7932    0.9095    0.8474       232
     B-MISC     0.5859    0.6554    0.6187       177
      B-ORG     0.8417    0.8446    0.8431       579
      B-PER     0.9791    0.9899    0.9844      1086
      E-LOC     0.7865    0.9052    0.8417       232
     E-MISC     0.6548    0.7288    0.6898       177
      E-ORG     0.8552    0.8566    0.8559       579
      E-PER     0.9782    0.9899    0.9840      1086
      I-LOC     0.5405    0.8000    0.6452        25
     I-MISC     0.4032    0.6410    0.4950        39
      I-ORG     0.7441    0.8633    0.7993       256
      I-PER     0.9565    0.9429    0.9496        70
          O     0.0000    0.0000    0.0000       327
      S-LOC     0.9273    0.9422    0.9347      1436
     S-MISC     0.8797    0.8495    0.8643       525
      S-ORG     0.9218    0.8826    0.9018      1082
      S-PER     0.9209    0.8776    0.8987       531

avg / total 


Evaluate model on the valid set
             precision    recall  f1-score   support

      B-LOC     0.9250    0.9487    0.9367       234
     B-MISC     0.8175    0.8016    0.8094       257
      B-ORG     0.8773    0.8578    0.8674       450
      B-PER     0.9808    0.9911    0.9859      1234
      E-LOC     0.9212    0.9487    0.9347       234
     E-MISC     0.8588    0.8521    0.8555       257
      E-ORG     0.9048    0.8867    0.8956       450
      E-PER     0.9807    0.9887    0.9847      1234
      I-LOC     0.7368    0.6087    0.6667        23
     I-MISC     0.7105    0.6067    0.6545        89
      I-ORG     0.9046    0.9136    0.9091       301
      I-PER     0.9464    0.7260    0.8217        73
          O     0.0000    0.0000    0.0000        99
      S-LOC     0.9756    0.9713    0.9734      1603
     S-MISC     0.9390    0.9023    0.9202       665
      S-ORG     0.9386    0.9439    0.9412       891
      S-PER     0.9223    0.9375    0.9299       608

avg / total

In [11]:
prediction_count=0
def predict(text):
    #         if prediction_count == 1:
    parameters['dataset_text_folder'] = os.path.join('..', 'data', 'temp')
    stats_graph_folder, _ = utils.create_stats_graph_folder(parameters)

    # Update the deploy folder, file, and dataset
    dataset_type = 'deploy'
    ### Delete all deployment data
    for filepath in glob.glob(os.path.join(parameters['dataset_text_folder'], '{0}*'.format(dataset_type))):
        if os.path.isdir(filepath):
            shutil.rmtree(filepath)
        else:
            os.remove(filepath)
    ### Create brat folder and file
    dataset_brat_deploy_folder = os.path.join(parameters['dataset_text_folder'], dataset_type)
    utils.create_folder_if_not_exists(dataset_brat_deploy_folder)
    dataset_brat_deploy_filepath = os.path.join(dataset_brat_deploy_folder, 'temp_{0}.txt'.format(
        str(prediction_count).zfill(5)))  # self._get_dataset_brat_deploy_filepath(dataset_brat_deploy_folder)
    with codecs.open(dataset_brat_deploy_filepath, 'w', 'UTF-8') as f:
        f.write(text)
    ### Update deploy filepaths
    dataset_filepaths, dataset_brat_folders = utils.get_valid_dataset_filepaths(parameters,
                                                                           dataset_types=[dataset_type])
    dataset_filepaths.update(dataset_filepaths)
    dataset_brat_folders.update(dataset_brat_folders)
    ### Update the dataset for the new deploy set
    dataset.update_dataset(dataset_filepaths, [dataset_type])

    # Predict labels and output brat
    output_filepaths = {}
    prediction_output = train.prediction_step(sess, dataset, dataset_type, model,
                                              transition_params_trained, stats_graph_folder,
                                              prediction_count, dataset_filepaths, parameters['tagging_format'])
    predictions , _, output_filepaths[dataset_type] = prediction_output
    
    print([dataset.index_to_label[prediction] for prediction in predictions])
    conll2brat.output_brat(output_filepaths, dataset_brat_folders, stats_graph_folder, overwrite=True)

    # Print and output result
    text_filepath = os.path.join(stats_graph_folder, 'brat', 'deploy',
                                 os.path.basename(dataset_brat_deploy_filepath))
    annotation_filepath = os.path.join(stats_graph_folder, 'brat', 'deploy', '{0}.ann'.format(
        utils.get_basename_without_extension(dataset_brat_deploy_filepath)))
    text2, entities = brat2conll.get_entities_from_brat(text_filepath, annotation_filepath, verbose=True)
    assert (text == text2)
    return entities


In [20]:
predict('my name is Phạm  Ngoc Linh')

Formatting deploy set from BRAT to CONLL... Done.
Converting CONLL from BIO to BIOES format... Done.
Predict labels for the deploy set
['O', 'O', 'O', 'O', 'B-PER', 'E-PER']
Formatting 000_deploy set from CONLL to BRAT... Done.

text:
my name is Phạm  Ngoc Linh

entity: {'id': 'T1', 'type': 'PER', 'start': 17, 'end': 26, 'text': 'Ngoc Linh'}





[{'end': 26, 'id': 'T1', 'start': 17, 'text': 'Ngoc Linh', 'type': 'PER'}]

In [21]:
predict("Hồ Chí Minh (/ˈhoʊ ˈtʃiː ˈmɪn/;[2] Vietnamese: [hò tɕǐ mīɲ] (About this sound listen), Saigon: [hò tɕǐ mɨ̄n] (About this sound listen); Chữ nôm: 胡志明; 19 May 1890 – 2 September 1969), born Nguyễn Sinh Cung,[3][4][5] also known as Nguyễn Tất Thành and Nguyễn Ái Quốc, was a Vietnamese Communist revolutionary leader who was Chairman and First Secretary of the Workers' Party of Vietnam. Hồ was also Prime Minister (1945–55) and President (1945–69) of the Democratic Republic of Vietnam (North Vietnam). He was a key figure in the foundation of the Democratic Republic of Vietnam in 1945, as well as the People's Army of Vietnam (PAVN) and the Việt Cộng (NLF or VC) during the Vietnam War.Hồ Chí Minh led the Việt Minh independence movement from 1941 onward, establishing the Communist-ruled Democratic Republic of Vietnam in 1945 and defeating the French Union in 1954 at the battle of Điện Biên Phủ. He officially stepped down from power in 1965 due to health problems. After the war, Saigon, the former capital of the Republic of Vietnam, was renamed Hồ Chí Minh City.Any description of Ho's life before he came to power in Vietnam is necessarily fraught with ambiguity. He is known to have used at least 50 and perhaps as many as 200 pseudonyms.[6] (Duiker says at least 75.)[7]:582 His place of birth and date of birth are products of academic consensus since neither is known with certainty. \"Official biographies and there are at least four, vary on names, dates, places and other hard facts. Unofficial biographies vary even more widely.")

Formatting deploy set from BRAT to CONLL... Done.
Converting CONLL from BIO to BIOES format... Done.
Predict labels for the deploy set
['B-ORG', 'I-ORG', 'E-ORG', 'O', 'B-ORG', 'I-ORG', 'E-ORG', 'O', 'S-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'E-MISC', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'E-PER', 'O', 'O', 'B-PER', 'E-PER', 'O', 'O', 'O', 'B-MISC', 'E-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'E-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'E-ORG', 'O', 'B-LOC', 'E-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'E-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'E-ORG', 'O', 'S-ORG', 'O',

[{'end': 11, 'id': 'T1', 'start': 0, 'text': 'Hồ Chí Minh', 'type': 'ORG'},
 {'end': 33,
  'id': 'T2',
  'start': 13,
  'text': '/ˈhoʊ ˈtʃiː ˈmɪn/;[2',
  'type': 'ORG'},
 {'end': 45, 'id': 'T3', 'start': 35, 'text': 'Vietnamese', 'type': 'MISC'},
 {'end': 213,
  'id': 'T4',
  'start': 188,
  'text': 'Nguyễn Sinh Cung,[3][4][5',
  'type': 'MISC'},
 {'end': 245, 'id': 'T5', 'start': 236, 'text': 'Tất Thành', 'type': 'PER'},
 {'end': 264, 'id': 'T6', 'start': 257, 'text': 'Ái Quốc', 'type': 'PER'},
 {'end': 292,
  'id': 'T7',
  'start': 272,
  'text': 'Vietnamese Communist',
  'type': 'MISC'},
 {'end': 383,
  'id': 'T8',
  'start': 335,
  'text': "First Secretary of the Workers' Party of Vietnam",
  'type': 'ORG'},
 {'end': 483,
  'id': 'T9',
  'start': 453,
  'text': 'Democratic Republic of Vietnam',
  'type': 'ORG'},
 {'end': 498,
  'id': 'T10',
  'start': 485,
  'text': 'North Vietnam',
  'type': 'LOC'},
 {'end': 576,
  'id': 'T11',
  'start': 546,
  'text': 'Democratic Republic of Vie